In [1]:
from reservoir_rls import *
from lorenz63 import *
from sklearn import preprocessing
from scipy.stats import sem
import cma
from scipy.stats import kstest, norm, shapiro
from multiprocessing import Pool
from min_func import *
from tqdm import tqdm
import pandas as pd



from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [6]:
data_length = 200000
step = 0.05
r_t = lambda x: r_t_const(x)
dxdt = lambda x,t,r_t: dxdt_lorenz_2scale(x,t,r_t)
data = get2ScaleLorenzData(data_length, r_t, dxdt, sample_tau = step) # Generate data with a constant rho value

# Set lengths for each train and validation data set
sync_length = 200
train_length = 800
pred_length = 300

scaled_data = data[:,:3]
scaled_data = np.ascontiguousarray(scaled_data)

In [7]:
num_nodes = 210
num_tests = 100
res_seed = 1
base_res = reservoir(3,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed) #Generate a reservoir
mask = ['input_weight', 'LM_regularization', 'forget']
x0 = np.array([5, 5])
# Set the minimization function. This function takes num_tests training
# and validation data sets and trains the reservoir to predict each validation set
# after being trained on the corresponding training set. The negative median valid
# time is returned to be minimized.
min_func = lambda x: vt_min_function_rls(scaled_data, x, mask, base_res.Win, base_res.A, \
     num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, \
     train_length = train_length, sampling = 'random', progress = True)
sigma = 2

In [9]:
result = min_func(np.array([5.07,2,0.5]))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.06it/s]


In [10]:
print(result)

-6.0


In [ ]:
opts = cma.CMAOptions()
opts.set('popsize',6*x0.size) # Set number of samples per generation
"""
Set bounds on parameters. IMPORTANT: The mean returned by cma-es is
the mean BEFORE the boundary function is applied, so the mean may not
lie in the domain set by bounds. To obtain the true sample mean requires 
downloading the cma-es package from github and editing one of the 
functions. Ask me if you need to do this.
"""
opts.set('bounds', [0,10]) 
opts.set('seed', 5) # Seed for the initial samples
"""
File where results are saved. IMPORTANT: Full covariance matrix is 
NOT saved, nor are the exact samples. If these need to be saved, one
will also have to download from github and make some edits. Again,
ask me.
"""
opts.set('verb_filenameprefix','cmaes_norm_res%d\\' % res_seed)
results = cma.fmin(min_func, x0, sigma, options = opts) # Run the algorithm